In [2]:
import os
import sys
import os
import logging
from easydict import EasyDict
import numpy as np
import random
import time
import datetime
from deap import tools
from collections import OrderedDict
from pprint import pprint
import json
import torch

sys.path.insert(0, '../')
from utils_kyy.utils_hr import make_random_graph_ex
from utils_kyy.create_toolbox_hr import create_toolbox_for_NSGA_RWNN_hr, evaluate_hr_full_train

import argparse




In [3]:

class rwns_train:
    def __init__(self, json_file):
        self.root = os.path.abspath(os.path.join(os.getcwd(), '..'))
        self.param_dir = os.path.join(self.root + '/parameters/', json_file)
        f = open(self.param_dir)
        params = json.load(f)
        pprint(params)
        self.name = params['NAME']

        ## toolbox params
        self.args_train = EasyDict(params['ARGS_TRAIN'])
        self.data_path = params['DATA_PATH']
        self.run_code = params['RUN_CODE']
        self.stage_pool_path = '../graph_pool' + '/' + self.run_code + '_' + self.name + '/'
        self.stage_pool_path_list = []

        self.log_path = '../logs/' + self.run_code + '_' + self.name + '/'
        # self.log_file_name : Initialize 부터 GA 진행상황 등 코드 전체에 대한 logging
        self.log_file_name = self.log_path + 'logging.log'
        # self.train_log_file_name : fitness (= flops, val_accuracy). 즉 GA history 를 저장 후, 나중에 사용하기 위한 logging.
        self.train_log_file_name = self.log_path + 'train_logging.log'

        if not os.path.exists(self.stage_pool_path):
            os.makedirs(self.stage_pool_path)
            for i in range(3):
                os.makedirs(self.stage_pool_path_list[i])

        if not os.path.isdir(self.log_path):
            os.makedirs(self.log_path)

        logging.basicConfig(filename=self.log_file_name, level=logging.INFO)
        logging.info('[Start] Rwns_train class is initialized.')
        logging.info('Start to write log.')

        self.num_graph = params['NUM_GRAPH']

        self.toolbox = self.create_toolbox()

        ## GA params
        self.pop_size = params['POP_SIZE']
        self.ngen = params['NGEN']
        self.cxpb = params['CXPB']
        self.mutpb = params['MUTPB']

        ## logs
        self.log = OrderedDict()
        self.log['hp'] = self.args_train
        self.train_log = OrderedDict()

    def create_toolbox(self):
        now = datetime.datetime.now()
        now_str = now.strftime('%Y-%m-%d %H:%M:%S')
        logging.info("[GA] Creating Tool Box " + now_str)
        return create_toolbox_for_NSGA_RWNN_hr(self.args_train, self.data_path, self.log_file_name)

    def train(self):
        ###################################
        # 1. Initialize the population.  (toolbox.population은 creator.Individual n개를 담은 list를 반환. (=> population)
        ###################################
        now = datetime.datetime.now()
        now_str = now.strftime('%Y-%m-%d %H:%M:%S')
        print("[GA] Initialion starts ...")
        logging.info("[GA] Initialion starts at " + now_str)
        init_start_time = time.time()

        pop = self.toolbox.population(n=self.pop_size)

        # fitness values (= accuracy, flops) 모음
        GA_history_list = []  # (choromosome, accuracy, flops) 이렇게 담아놓기
        # e.g.  [ [[1,3,4], 40, 1000], [[2,6,10], 30%, 2000], ...  ]

        ###################################
        # 2. Evaluate the population (with an invalid fitness)
        ###################################
        invalid_ind = [ind for ind in pop]
        for idx, ind in enumerate(invalid_ind):
            fitness, ind_model = evaluate_hr_full_train(ind[0], args_train=self.args_train,
                                             data_path=self.data_path, log_file_name=self.log_file_name)
            ind.fitness.values = fitness
            GA_history_list.append([ind, fitness])

        ## log 기록 - initialize (= 0th generation)
        self.train_log[0] = GA_history_list

        self.save_log()

        # This is just to assign the crowding distance to the individuals
        # no actual selection is done
        pop = self.toolbox.select(pop, len(pop))

        now = datetime.datetime.now()
        now_str = now.strftime('%Y-%m-%d %H:%M:%S')
        print("Initialization is finished at", now_str)
        logging.info("Initialion is finished at " + now_str)

        init_time = time.time() - init_start_time
        logging.info("Initialization time = " + str(init_time) + "s")
        print()

        ###################################
        # 3. Begin GA
        ###################################
        # Begin the generational process
        for gen in range(1, self.ngen):
            ##### 3.1. log 기록
            now = datetime.datetime.now()
            now_str = now.strftime('%Y-%m-%d %H:%M:%S')
            print("#####", gen, "th generation starts at", now_str)
            logging.info("#####" + str(gen) + "th generation starts at" + now_str)

            start_gen = time.time()

            ##### 3.2. Offspring pool 생성 후, crossover(=mate) & mutation
            # Vary the population
            offspring = tools.selTournamentDCD(pop, len(pop))
            offspring = [self.toolbox.clone(ind) for ind in offspring]

            # ::2, 1::2 즉, 짝수번째 크로모좀과 홀수번쨰 크로모좀들 차례로 선택하면서 cx, mut 적용
            # e.g. 0번, 1번 ind를 cx, mut   // 2번, 3번 ind를 cx, mut // ...
            for ind1, ind2 in zip(offspring[::2], offspring[1::2]):
                if random.random() <= self.cxpb:
                    self.toolbox.mate(ind1, ind2)

                self.toolbox.mutate(ind1, indpb=self.mutpb)
                self.toolbox.mutate(ind2, indpb=self.mutpb)
                del ind1.fitness.values, ind2.fitness.values

            ##### 3.3. Evaluation
            # Evaluate the individuals with an invalid fitness
            print("\t Evaluation...")
            start_time = time.time()

            # fitness values (= accuracy, flops) 모음
            GA_history_list = []

            invalid_ind = [ind for ind in offspring]

            for idx, ind in enumerate(invalid_ind):
                fitness, ind_model = evaluate_hr_full_train(ind[0], args_train=self.args_train,
                                                 stage_pool_path_list=self.stage_pool_path_list,
                                                 data_path=self.data_path, log_file_name=self.log_file_name)
                # <= evaluate() returns  (-prec, flops), NN_model

                ind.fitness.values = fitness
                GA_history_list.append([ind, fitness])

            ## log 기록
            self.train_log[gen] = GA_history_list

            self.save_log()

            eval_time_for_one_generation = time.time() - start_time
            print("\t Evaluation ends (Time : %.3f)" % eval_time_for_one_generation)

            ##### Select the next generation population
            pop = self.toolbox.select(pop + offspring, self.pop_size)

            gen_time = time.time() - start_gen
            print('\t [gen_time: %.3fs]' % gen_time, gen, 'th generation is finished.')

            logging.info('\t Gen [%03d/%03d] -- evals: %03d, evals_time: %.4fs, gen_time: %.4fs' % (
                gen, self.ngen, len(invalid_ind), eval_time_for_one_generation, gen_time))

    ## Save Log
    def save_log(self):
        ## 필요한 log 추후 정리하여 추가
        self.log['train_log'] = self.train_log

        with open(self.train_log_file_name, 'w', encoding='utf-8') as make_file:
            json.dump(self.log, make_file, ensure_ascii=False, indent='\t')



### Debuggingg

In [4]:
json_file = '../parameters/hr_unit.json'

trainer = rwns_train(json_file=json_file)

{'ARGS_TRAIN': {'base_lr': 0.2,
                'batch_size': 100,
                'data': 'CIFAR10',
                'epochs': 1,
                'graycode': False,
                'hr': True,
                'input_dim': 3,
                'lr_mode': 'cosine',
                'momentum': 0.9,
                'nsize': 15,
                'num_classes': 10,
                'print_freq': 100,
                'targetlr': 0.0,
                'warmup_epochs': 0,
                'warmup_lr': 0.0,
                'warmup_mode': 'linear',
                'weight_decay': 5e-05,
                'workers': 64},
 'CXPB': 0.5,
 'DATA_PATH': 'D:/data/',
 'EPOCH_WEIGHT': 10,
 'MUTPB': 0.1,
 'NAME': 'hr_unit',
 'NGEN': 30,
 'NUM_GRAPH': 128,
 'POP_SIZE': 8,
 'RUN_CODE': 'main_hr'}


In [5]:
trainer.create_toolbox()
trainer.train()

C:\Users\82103\Anaconda3\envs\newrw\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
C:\Users\82103\Anaconda3\envs\newrw\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


[GA] Initialion starts ...
length of one individual 105
15
15
15
gmat value [[7. 7. 2. 7. 1. 7. 7. 7. 7. 0. 7. 7. 3. 7. 7.]
 [7. 7. 2. 7. 7. 7. 7. 7. 7. 7. 0. 1. 7. 7. 7.]
 [7. 7. 7. 7. 7. 7. 7. 7. 7. 7. 3. 6. 7. 6. 7.]
 [7. 7. 7. 7. 6. 7. 7. 4. 2. 7. 1. 7. 7. 7. 3.]
 [7. 7. 7. 7. 7. 3. 7. 7. 7. 7. 7. 7. 7. 7. 4.]
 [7. 7. 7. 7. 7. 7. 7. 7. 7. 7. 6. 1. 7. 3. 7.]
 [7. 7. 7. 7. 7. 7. 7. 7. 4. 7. 7. 7. 7. 0. 7.]
 [7. 7. 7. 7. 7. 7. 7. 7. 2. 7. 7. 7. 7. 3. 7.]
 [7. 7. 7. 7. 7. 7. 7. 7. 7. 7. 7. 7. 7. 7. 7.]
 [7. 7. 7. 7. 7. 7. 7. 7. 7. 7. 7. 5. 6. 7. 1.]
 [7. 7. 7. 7. 7. 7. 7. 7. 7. 7. 7. 2. 7. 7. 7.]
 [7. 7. 7. 7. 7. 7. 7. 7. 7. 7. 7. 7. 5. 7. 7.]
 [7. 7. 7. 7. 7. 7. 7. 7. 7. 7. 7. 7. 7. 7. 7.]
 [7. 7. 7. 7. 7. 7. 7. 7. 7. 7. 7. 7. 7. 7. 5.]
 [7. 7. 7. 7. 7. 7. 7. 7. 7. 7. 7. 7. 7. 7. 7.]]
output_nodes value [8, 12, 14]
gmat value [[7. 7. 7. 0. 3. 3. 0. 7. 7. 7. 7. 7. 7. 3. 0.]
 [7. 7. 6. 3. 7. 7. 7. 7. 7. 7. 0. 7. 7. 7. 7.]
 [7. 7. 7. 3. 7. 7. 7. 2. 0. 7. 7. 7. 7. 7. 7.]
 [7. 7. 7. 7. 7. 

BrokenPipeError: [Errno 32] Broken pipe

In [3]:
from deap import base, creator
from deap import tools


import random
from itertools import repeat
from collections import Sequence

# For evaluate function --------------------------
import glob
from easydict import EasyDict

import numpy as np

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn    # for hardware tunning (cudnn.benchmark = True)

import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

from thop import profile
from thop import clever_format

import logging

# Gray code package
from utils_kyy.utils_graycode_v2 import *

# custom package in utils_kyy
from utils_kyy.utils_graph import load_graph
from utils_kyy.utils_hr import *
from utils_kyy.models_hr import RWNN
from utils_kyy.train_validate import train, validate, train_AMP
from utils_kyy.lr_scheduler import LRScheduler
from torchsummary import summary


def make_random_graph_ex(num_graph):
    # 해당 stage_pool_path에 stage가 꽉 차있는지 확인

    #Nodes = random.randint(20, 40)  # => [Nodes 값 수정 시 주의] 아래 K값은 Nodes보다 작아야함.
    Nodes = num_graph
    graph_model = 'WS'
    K = random.randint(4, Nodes - 10)  # [min, max] // WS에서는 K nearest. 따라서, 4 ~ 30 random 선택 하도록
    P = round(random.uniform(0.25, 0.75), 2)  # 소수 둘째자리까지만 나오도록 반올림 => 결과 e.g. 0.75

    args = EasyDict({'graph_model': graph_model, 'K': K, 'P': P})
    print(args)
    P_str = str(P)[0] + str(P)[2:]  # 0.75 => 075

    graph = build_graph(Nodes, args)

    return graph

def init_graph(nsize):
    # nsize : number of nodes

    individuals = []

    for i in range(3):  # For s stages
        grph_ex = make_random_graph_ex(nsize)  # Make random graph using WS

        ## G matrix 구하기
        nds, inds, onds = get_graph_info(grph_ex)
        g_mat = get_g_matrix(nds)
        
#         print("Gmat")
#         print(g_mat)
        individual = gmat2ind(g_mat)
        print(individual)
        individuals.extend(individual)

    return individuals


def gmat2ind(g_mat):
    nsize = g_mat.shape[0]
    g_encode = []

    for idx, col_idx in enumerate(range(1, nsize)):
        row_idx = idx + 1
        g_encode.extend(g_mat[:row_idx, col_idx].tolist())

    return g_encode


def ind2gmat(g_encode, nsize):

    decode_mat = np.ones((nsize, nsize))

    # to initlize 7 (disconnected)
    decode_mat = decode_mat * 7

    row_idx = 0
    col_idx = 1
    for idx, v in enumerate(g_encode):
        decode_mat[row_idx][col_idx] = v
        if row_idx + 1 == col_idx:
            col_idx += 1
            row_idx = 0
        else:
            row_idx += 1

    return decode_mat

In [4]:
temp = init_graph(15)

{'graph_model': 'WS', 'K': 5, 'P': 0.6}
[7, 7, 3, 7, 3, 7, 7, 7, 6, 7, 7, 7, 7, 7, 7, 7, 7, 4, 7, 1, 0, 7, 7, 3, 7, 5, 4, 7, 6, 7, 7, 7, 5, 6, 7, 7, 1, 7, 7, 7, 7, 7, 7, 7, 4, 4, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 2, 7, 7, 5, 7, 7, 7, 7, 2, 7, 7, 7, 6, 7, 7, 7, 7, 7, 7, 7, 7, 4, 7, 7, 7, 7, 7, 7, 2, 7, 7, 1, 7, 4, 0, 7, 7, 0, 5, 7, 7, 3, 7, 5, 7, 7, 6, 0]
{'graph_model': 'WS', 'K': 5, 'P': 0.61}
[0, 7, 4, 7, 7, 2, 7, 7, 7, 7, 5, 7, 7, 1, 7, 6, 7, 7, 7, 4, 6, 7, 0, 7, 7, 7, 7, 5, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 0, 4, 7, 7, 6, 3, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 1, 7, 3, 5, 7, 3, 7, 7, 7, 0, 7, 6, 7, 4, 7, 1, 3, 7, 7, 7, 1, 0, 0, 7, 7, 7, 7, 0, 7, 0, 7, 5, 7, 7, 7, 6, 7, 7, 7, 7, 7, 7, 7, 7, 3]
{'graph_model': 'WS', 'K': 5, 'P': 0.59}
[6, 7, 7, 7, 7, 7, 7, 7, 7, 3, 7, 1, 7, 7, 2, 7, 7, 7, 5, 7, 7, 1, 7, 7, 7, 4, 7, 6, 7, 2, 7, 7, 7, 6, 1, 5, 7, 7, 4, 7, 7, 7, 7, 0, 7, 7, 7, 7, 1, 7, 7, 7, 7, 7, 1, 7, 1, 1, 7, 7, 7, 7, 7, 7, 7, 0, 7, 7, 7, 7, 7, 7, 7, 7, 7, 3, 0, 4, 7, 1, 7, 7,

In [5]:
ind2gmat(temp[:105],15)

array([[7., 7., 7., 7., 7., 7., 7., 7., 6., 1., 4., 7., 7., 4., 0.],
       [7., 7., 3., 3., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7.],
       [7., 7., 7., 7., 6., 7., 4., 3., 7., 7., 7., 2., 7., 7., 7.],
       [7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 6., 7., 0.],
       [7., 7., 7., 7., 7., 7., 1., 5., 5., 7., 7., 7., 7., 7., 5.],
       [7., 7., 7., 7., 7., 7., 0., 4., 6., 7., 7., 5., 7., 7., 7.],
       [7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7.],
       [7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 2., 3.],
       [7., 7., 7., 7., 7., 7., 7., 7., 7., 4., 7., 7., 7., 7., 7.],
       [7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 5.],
       [7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 2., 7., 1., 7.],
       [7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7.],
       [7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 4., 6.],
       [7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 0.],
       [7., 7., 7., 7., 7., 7., 7.

In [6]:
len(temp)

315

In [8]:
json_file = '../parameters/hr_unit.json'

param_dir = os.path.join('../parameters/', json_file)
f = open(param_dir)
params = json.load(f)
pprint(params)
name = params['NAME']

## toolbox params
args_train = EasyDict(params['ARGS_TRAIN'])

{'ARGS_TRAIN': {'base_lr': 0.2,
                'batch_size': 24,
                'data': 'CIFAR10',
                'epochs': 1,
                'graycode': False,
                'hr': True,
                'input_dim': 3,
                'lr_mode': 'cosine',
                'momentum': 0.9,
                'nsize': 15,
                'num_classes': 10,
                'print_freq': 100,
                'targetlr': 0.0,
                'warmup_epochs': 0,
                'warmup_lr': 0.0,
                'warmup_mode': 'linear',
                'weight_decay': 5e-05,
                'workers': 12},
 'CXPB': 0.5,
 'DATA_PATH': 'D:/data/',
 'EPOCH_WEIGHT': 10,
 'MUTPB': 0.1,
 'NAME': 'hr_unit',
 'NGEN': 30,
 'POP_SIZE': 4,
 'RUN_CODE': 'main_hr'}


In [12]:
# => Min ( -val_accuracy(top_1),  flops )
creator.create('FitnessMin', base.Fitness, weights=(-1.0, -1.0))  # name, base (class), attribute //
creator.create('Individual', list, fitness=creator.FitnessMin)  # creator.FitnessMaxMin attribute로 가짐

#####################################
# Initialize the toolbox
#####################################
toolbox = base.Toolbox()

if args_train.hr:
    ## Nsize -> individual size
    nsize = args_train.nsize
    BOUND_LOW = 0
    BOUND_UP = 7  # Operation 종류
    # toolbox.register('attr_int', random.randint, BOUND_LOW, BOUND_UP)

    toolbox.register('init_graph',init_graph, nsize)
    # input : nsize
    # output : nsize -> Graph -> Gmat -> indivdual로 연계되게

else :
    print("change hr to True")

## HR 일 때는 WS로 initiat 하고 싶은데 이 부분 좀 생각이 필요할듯
toolbox.register('individual', tools.initIterate,
                 creator.Individual, toolbox.init_graph)

toolbox.register('population', tools.initRepeat,
                 list, toolbox.individual)  # n은 생략함. toolbox.population 함수를 뒤에서 실행할 때 넣어줌.

# crossover

if args_train.hr:
    toolbox.register('mate', cxhr)
else:
    toolbox.register('mate', tools.cxTwoPoint)  # crossover


C:\Users\82103\Anaconda3\envs\newrw\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
C:\Users\82103\Anaconda3\envs\newrw\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


In [13]:
data_path=params['DATA_PATH']
log_file_name= '/temp'

In [14]:
now = datetime.datetime.now()
now_str = now.strftime('%Y-%m-%d %H:%M:%S')
print("[GA] Initialion starts ...")
logging.info("[GA] Initialion starts at " + now_str)
init_start_time = time.time()

pop = toolbox.population(n=10)


[GA] Initialion starts ...
{'graph_model': 'WS', 'K': 4, 'P': 0.63}
[0, 7, 1, 4, 7, 2, 7, 7, 7, 2, 7, 7, 1, 7, 0, 7, 7, 7, 7, 7, 7, 7, 7, 7, 1, 7, 7, 7, 7, 7, 7, 7, 7, 7, 2, 7, 7, 7, 7, 7, 7, 7, 3, 3, 0, 7, 6, 3, 7, 7, 7, 6, 7, 1, 7, 6, 7, 6, 7, 5, 7, 7, 3, 7, 7, 7, 7, 7, 7, 3, 7, 7, 7, 7, 5, 7, 7, 7, 7, 7, 7, 7, 6, 7, 7, 7, 3, 7, 7, 4, 7, 7, 3, 7, 7, 7, 7, 4, 7, 7, 7, 7, 6, 4, 3]
{'graph_model': 'WS', 'K': 4, 'P': 0.74}
[1, 3, 5, 7, 7, 0, 7, 7, 7, 7, 7, 7, 7, 2, 6, 7, 7, 7, 7, 5, 1, 7, 7, 0, 7, 7, 6, 7, 7, 7, 3, 7, 7, 7, 5, 7, 7, 7, 7, 5, 7, 6, 7, 7, 7, 7, 7, 3, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 4, 7, 3, 7, 7, 7, 7, 7, 7, 4, 7, 7, 7, 7, 2, 5, 4, 7, 4, 7, 7, 7, 2, 7, 7, 7, 7, 1, 7, 7, 7, 7, 6, 3, 7, 7, 7, 3, 7, 7, 5, 7, 7, 7, 2, 1, 7]
{'graph_model': 'WS', 'K': 5, 'P': 0.66}
[7, 7, 7, 3, 7, 0, 7, 7, 7, 7, 7, 7, 7, 7, 5, 7, 7, 7, 7, 7, 5, 7, 0, 7, 1, 1, 1, 3, 6, 7, 7, 7, 7, 7, 0, 7, 7, 7, 7, 7, 3, 7, 7, 7, 7, 7, 7, 2, 7, 7, 7, 7, 7, 4, 1, 3, 7, 7, 4, 7, 7, 7, 7, 7, 7, 7, 7, 4, 7, 7, 5, 4, 7

In [16]:
len(pop)

10

In [70]:
pop = toolbox.population(n=10)


{'graph_model': 'WS', 'K': 4, 'P': 0.29}
Gmat
[[7 7 1 5 7 7 7 7 7 7 3 3 7 5 5]
 [7 7 5 5 7 7 7 7 7 7 7 7 7 7 7]
 [7 7 7 0 7 7 7 2 7 4 7 7 7 7 2]
 [7 7 7 7 0 7 5 7 7 6 7 7 7 7 7]
 [7 7 7 7 7 2 0 7 7 7 7 7 7 7 7]
 [7 7 7 7 7 7 1 7 7 7 7 7 7 7 2]
 [7 7 7 7 7 7 7 7 0 7 7 7 7 7 7]
 [7 7 7 7 7 7 7 7 4 1 7 7 7 7 7]
 [7 7 7 7 7 7 7 7 7 1 7 7 7 3 7]
 [7 7 7 7 7 7 7 7 7 7 7 7 7 7 7]
 [7 7 7 7 7 7 7 7 7 7 7 0 3 7 7]
 [7 7 7 7 7 7 7 7 7 7 7 7 0 7 7]
 [7 7 7 7 7 7 7 7 7 7 7 7 7 6 1]
 [7 7 7 7 7 7 7 7 7 7 7 7 7 7 5]
 [7 7 7 7 7 7 7 7 7 7 7 7 7 7 7]]
[7, 1, 5, 5, 5, 0, 7, 7, 7, 0, 7, 7, 7, 7, 2, 7, 7, 7, 5, 0, 1, 7, 7, 2, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 0, 4, 7, 7, 4, 6, 7, 7, 7, 1, 1, 3, 7, 7, 7, 7, 7, 7, 7, 7, 7, 3, 7, 7, 7, 7, 7, 7, 7, 7, 7, 0, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 3, 0, 5, 7, 7, 7, 7, 7, 7, 7, 3, 7, 7, 7, 6, 5, 7, 2, 7, 7, 2, 7, 7, 7, 7, 7, 7, 1, 5]
{'graph_model': 'WS', 'K': 4, 'P': 0.57}
Gmat
[[7 6 7 7 7 1 7 7 7 7 7 7 7 6 7]
 [7 7 7 5 7 7 7 7 5 3 7 7 7 7 6]
 [7 7 7 6 6 7 0 7 7 7 7 7 7 7 

 [7 7 7 7 7 7 7 7 7 7 7 7 7 7 7]]
[7, 7, 4, 7, 5, 0, 7, 7, 7, 7, 7, 7, 7, 7, 2, 7, 7, 7, 7, 0, 6, 7, 7, 7, 5, 7, 4, 5, 7, 7, 7, 7, 7, 7, 2, 7, 6, 7, 7, 7, 7, 7, 7, 7, 7, 3, 7, 7, 3, 7, 7, 7, 2, 0, 3, 7, 5, 7, 7, 7, 7, 6, 7, 3, 4, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 6, 7, 7, 5, 7, 6, 7, 7, 7, 7, 2, 7, 7, 2, 5, 7, 7, 7, 7, 7, 1, 7, 7, 7, 7, 5, 7, 3, 7]
{'graph_model': 'WS', 'K': 4, 'P': 0.58}
Gmat
[[7 7 6 2 7 7 7 7 7 7 6 1 4 7 2]
 [7 7 3 7 7 7 7 7 7 5 7 7 6 7 4]
 [7 7 7 5 7 7 7 7 7 7 7 7 7 7 4]
 [7 7 7 7 7 7 0 7 7 5 3 7 7 7 7]
 [7 7 7 7 7 7 3 7 7 2 7 7 7 7 7]
 [7 7 7 7 7 7 2 4 7 7 7 7 7 7 7]
 [7 7 7 7 7 7 7 0 5 7 7 7 7 7 7]
 [7 7 7 7 7 7 7 7 7 5 7 7 7 0 7]
 [7 7 7 7 7 7 7 7 7 1 7 7 2 7 7]
 [7 7 7 7 7 7 7 7 7 7 7 2 7 7 7]
 [7 7 7 7 7 7 7 7 7 7 7 2 7 7 7]
 [7 7 7 7 7 7 7 7 7 7 7 7 7 2 7]
 [7 7 7 7 7 7 7 7 7 7 7 7 7 2 7]
 [7 7 7 7 7 7 7 7 7 7 7 7 7 7 4]
 [7 7 7 7 7 7 7 7 7 7 7 7 7 7 7]]
[7, 6, 3, 2, 7, 5, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 0, 3, 2, 7, 7, 7, 7, 7, 4, 0, 7, 7, 7, 7, 7, 7, 5, 7,

 [7 7 7 7 7 7 7 7 7 7 7 7 7 7 7]]
[3, 7, 7, 7, 3, 7, 7, 7, 7, 7, 7, 7, 7, 0, 7, 7, 6, 7, 1, 0, 3, 7, 5, 7, 7, 5, 7, 5, 5, 7, 7, 7, 1, 7, 2, 7, 7, 7, 7, 7, 2, 7, 7, 0, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 1, 7, 7, 7, 4, 7, 7, 0, 7, 7, 7, 7, 7, 4, 7, 0, 7, 4, 7, 7, 7, 7, 3, 7, 7, 6, 7, 7, 3, 1, 7, 7, 6, 7, 7, 4, 1, 7, 7, 3, 7, 7, 7, 1]
{'graph_model': 'WS', 'K': 4, 'P': 0.42}
Gmat
[[7 1 7 7 7 0 5 7 7 7 7 7 7 3 6]
 [7 7 7 0 0 7 7 7 7 7 3 7 7 7 7]
 [7 7 7 7 2 7 7 7 7 4 1 7 7 7 7]
 [7 7 7 7 6 6 7 7 7 7 7 7 7 7 7]
 [7 7 7 7 7 7 2 7 7 7 7 7 7 7 7]
 [7 7 7 7 7 7 7 1 7 6 7 7 7 2 7]
 [7 7 7 7 7 7 7 2 7 2 7 7 7 7 7]
 [7 7 7 7 7 7 7 7 0 7 7 7 7 7 7]
 [7 7 7 7 7 7 7 7 7 4 3 7 7 7 7]
 [7 7 7 7 7 7 7 7 7 7 6 7 3 7 7]
 [7 7 7 7 7 7 7 7 7 7 7 7 1 7 7]
 [7 7 7 7 7 7 7 7 7 7 7 7 2 3 7]
 [7 7 7 7 7 7 7 7 7 7 7 7 7 2 0]
 [7 7 7 7 7 7 7 7 7 7 7 7 7 7 0]
 [7 7 7 7 7 7 7 7 7 7 7 7 7 7 7]]
[1, 7, 7, 7, 0, 7, 7, 0, 2, 6, 0, 7, 7, 6, 7, 5, 7, 7, 7, 2, 7, 7, 7, 7, 7, 7, 1, 2, 7, 7, 7, 7, 7, 7, 7, 0,

 [7 7 7 7 7 7 7 7 7 7 7 7 7 7 7]]
[7, 7, 7, 7, 7, 7, 7, 7, 6, 3, 4, 7, 7, 4, 2, 6, 7, 7, 7, 5, 7, 7, 7, 7, 7, 7, 7, 7, 2, 5, 5, 3, 7, 5, 7, 0, 7, 7, 7, 7, 3, 7, 7, 3, 7, 7, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 0, 7, 7, 7, 7, 1, 7, 7, 6, 7, 7, 3, 7, 0, 7, 7, 7, 7, 2, 7, 1, 1, 7, 7, 7, 7, 7, 7, 7, 7, 7, 1, 7, 6, 7, 2, 7, 7, 7, 7, 7, 0, 7, 7, 7, 7, 7, 4, 2]


In [74]:
pop[0]

[[7,
  1,
  5,
  5,
  5,
  0,
  7,
  7,
  7,
  0,
  7,
  7,
  7,
  7,
  2,
  7,
  7,
  7,
  5,
  0,
  1,
  7,
  7,
  2,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  0,
  4,
  7,
  7,
  4,
  6,
  7,
  7,
  7,
  1,
  1,
  3,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  3,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  0,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  3,
  0,
  5,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  3,
  7,
  7,
  7,
  6,
  5,
  7,
  2,
  7,
  7,
  2,
  7,
  7,
  7,
  7,
  7,
  7,
  1,
  5,
  6,
  7,
  7,
  7,
  5,
  6,
  7,
  7,
  6,
  0,
  1,
  7,
  7,
  7,
  7,
  7,
  7,
  0,
  7,
  3,
  4,
  7,
  7,
  7,
  7,
  2,
  4,
  7,
  7,
  5,
  7,
  7,
  7,
  7,
  7,
  3,
  7,
  3,
  7,
  7,
  7,
  7,
  4,
  4,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  1,
  7,
  7,
  7,
  3,
  7,
  7,
  4,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  1,
  7,
  2,
  5,
  6,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  3,
  7,
  0,
  7,
  7,
  6,
  7,
  7,


### Fixing population
* need to squeeze the output
* list of list -> list
* initRepeat -> initIterate

#### Cross over and mutate

In [ ]:

def cxhr(ind1, ind2):

    ## individual 받고 -> 3등분 -> 3등분 안에서만 시행
    new_ind1 = []
    new_ind2 = []


    one_len = len(ind1)/3

    for i in range(3):
        temp_ind1 = ind1[one_len*i : one_len* (i+1)]
        temp_ind2 = ind2[one_len*i : one_len* (i+1)]

        x1, x2 = cxOnePoint(temp_ind1,temp_ind2)

        new_ind1.extend(x1)
        new_ind2.extend(x2)

    return  new_ind1, new_ind2


In [1]:
import torch.cuda as cutorch

for i in range(cutorch.device_count()):
     if cutorch.getMemoryUsage(i) > MEM: 
         opts.gpuID = i
         break

AttributeError: module 'torch.cuda' has no attribute 'getMemoryUsage'

In [2]:
import subprocess

def get_gpu_memory_map():
    """Get the current gpu usage.

    Returns
    -------
    usage: dict
        Keys are device ids as integers.
        Values are memory usage as integers in MB.
    """
    result = subprocess.check_output(
        [
            'nvidia-smi', '--query-gpu=memory.used',
            '--format=csv,nounits,noheader'
        ], encoding='utf-8')
    # Convert lines into a dictionary
    gpu_memory = [int(x) for x in result.strip().split('\n')]
    gpu_memory_map = dict(zip(range(len(gpu_memory)), gpu_memory))
    return gpu_memory_map

In [3]:
get_gpu_memory_map()

TypeError: __init__() got an unexpected keyword argument 'encoding'

In [9]:
import torch
torch.cuda.get_device_properties(3).total_memory


12788498432

In [10]:
torch.cuda.empty_cache()

In [12]:
torch.cuda.memory_summary(device=None, abbreviated=False)

AttributeError: module 'torch.cuda' has no attribute 'memory_summary'

In [13]:
torch.cuda.memory_snapshot()

AttributeError: module 'torch.cuda' has no attribute 'memory_snapshot'

In [15]:
torch.cuda.memory_allocated(1)

0

In [16]:
    torch.cuda.empty_cache()
